In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('commands.json').read()
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

94 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [7]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [8]:
# Training Data Creation
training = []
# Define Output Array
output_empty = [0] * len(classes)
# Training data as Bag of words for respective sentence
for doc in documents:
    # Bag of words Intialization
    bag = []
    # tokenized word list as pattern
    pattern_words = doc[0]
    # lemmatize the wordsand define the meaning
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Creation of bag of words array with 1, if it matches in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for each pattern
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# Conversion to  np.array
random.shuffle(training)
training = np.array(training)
# create train and test dataset
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\ensar\AppData\Local\Temp/ipykernel_16352/2801411539.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [9]:
# Create model - 3 layers neural network
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model.
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

D:\Software\Anaconda3\envs\Sara\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
19/19 [==============================] - 2s 3ms/step - loss: 2.2234 - accuracy: 0.0851 
Epoch 2/200
19/19 [==============================] - 0s 4ms/step - loss: 2.0472 - accuracy: 0.2766
Epoch 3/200
19/19 [==============================] - 0s 5ms/step - loss: 1.9609 - accuracy: 0.3511
Epoch 4/200
19/19 [==============================] - 0s 4ms/step - loss: 1.6974 - accuracy: 0.5426
Epoch 5/200
19/19 [==============================] - 0s 5ms/step - loss: 1.4514 - accuracy: 0.5213
Epoch 6/200
19/19 [==============================] - 0s 5ms/step - loss: 1.2288 - accuracy: 0.6702
Epoch 7/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0162 - accuracy: 0.7021
Epoch 8/200
19/19 [==============================] - 0s 5ms/step - loss: 0.8163 - accuracy: 0.7021
Epoch 9/200
19/19 [==============================] - 0s 4ms/step - loss: 0.6919 - accuracy: 0.8191
Epoch 10/200
19/19 [==============================] - 0s 4ms/step - loss: 0.5984 - accuracy: 0.8191
Epoch 11

19/19 [==============================] - 0s 1ms/step - loss: 0.0144 - accuracy: 1.0000
Epoch 84/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 85/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0181 - accuracy: 0.9894
Epoch 86/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0209 - accuracy: 0.9894
Epoch 87/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 88/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0450 - accuracy: 0.9894
Epoch 89/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0218 - accuracy: 0.9894
Epoch 90/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0281 - accuracy: 0.9894
Epoch 91/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0181 - accuracy: 1.0000
Epoch 92/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0487 - accuracy: 0.9681
Epoch 93/200


19/19 [==============================] - 0s 3ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 165/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 166/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0111 - accuracy: 0.9894
Epoch 167/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 168/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 169/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0230 - accuracy: 0.9894
Epoch 170/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 171/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0216 - accuracy: 1.0000
Epoch 172/200
19/19 [==============================] - 0s 3ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 173/200
19/19 [==============================] - 0s 1ms/step - loss: 0.0034 - accuracy: 1.0000
Epoc